# Build Recurrent Neural Network

In [2]:
import numpy as np
from rnn_utils import *

### RNN single cell

#### Forward

$ a^{<t>} = tanh(W_{ax}x^{<t>} + W_{aa}a^{<t-1>} + b_a) $

$ \hat{y}^{<t>} = softmax(W_{ya}a^{<t>} + b_y) $


In [22]:
def rnn_cell_forward(xt, a_prev, params):
    """
    Implement a single forward step of the RNN-cell.
    
    Arguments:
        xt -- Input data at timesamp "t", numpy array of shape (n_x, m).
        a_prev -- Hidden state at timestep "t-1", numpy array of shape (n_a, m).
        params -- python dictionary containing:
            Wax -- Weight matrix multiplying the input, numpy array of shape (n_a, n_x).
            Waa -- Weight matrix multiplying the hidden state, numpy array of shape (n_a, n_a).
            Wya -- Weight matrix relating the hidden-state to the output, numpy array of shape (n_y, n_a).
            ba -- Bias, numpy array of shape (n_a, 1).
            by -- Bias relating the hidden-state to the output, numpy array of shape (n_y, 1).
            
    Returns:
        a_next -- Next hidden-state, numpy array of shape (n_a, m).
        yt_pred -- Prediction at timestamp "t", numpy array of shape (n_y, m).
        caches -- tuple of values needed for the backward pass, contains (a_next, a_prev, xt, params).
    """
    
    # Retrieve parameters form "params"
    Wax = params["Wax"]
    Waa = params["Waa"]
    Wya = params["Wya"]
    ba = params["ba"]
    by = params["by"]
    
    a_next = np.tanh(np.dot(Wax, xt) + np.dot(Waa, a_prev) + ba)
    yt_pred = softmax(np.dot(Wya, a_next) + by)
    
    cache = (a_next, a_prev, xt, params)
    return a_next, yt_pred, cache

In [31]:
np.random.seed(1)
xt = np.random.randn(3, 10)
a_prev = np.random.randn(5, 10)
Waa = np.random.randn(5, 5)
Wax = np.random.randn(5, 3)
Wya = np.random.randn(2, 5)
ba = np.random.randn(5, 1)
by = np.random.randn(2, 1)

parameters = {
    "Waa": Waa,
    "Wax": Wax,
    "Wya": Wya,
    "ba": ba,
    "by": by
}

a_next, yt_pred, cache = rnn_cell_forward(xt, a_prev, parameters)
print("a_next[4] = {}".format(a_next[4]))
print("a_next.shape = {}".format(a_next.shape))
print("yt_pred[1] = {}".format(yt_pred[1]))
print("yt_pred.shape = {}".format(yt_pred.shape))

a_next[4] = [ 0.59584544  0.18141802  0.61311866  0.99808218  0.85016201  0.99980978
 -0.18887155  0.99815551  0.6531151   0.82872037]
a_next.shape = (5, 10)
yt_pred[1] = [0.9888161  0.01682021 0.21140899 0.36817467 0.98988387 0.88945212
 0.36920224 0.9966312  0.9982559  0.17746526]
yt_pred.shape = (2, 10)


### Backward

$ a^{<t>} = tanh(W_{ax}x^{<t>} + W_{aa}a^{<t-1>} + b) $

$ \frac{\partial tanh(x)}{\partial x} = 1 - tanh(x)^{2} $

$ \frac{\partial a^{<t>}}{\partial W_{ax}} = (1 - tanh(W_{ax}x^{<t>} + W_{aa}a^{<t-1>} + b)^2)x^{<t>T} = \partial tanh(x)x^{<t>T} $

$ \frac{\partial a^{<t>}}{\partial W_{aa}} = (1 - tanh(W_{ax}x^{<t>} + W_{aa}a^{<t-1>} + b)^2)x^{<t-1>T} = \partial tanh(x)x^{<t-1>T} $

$ \frac{\partial a^{<t>}}{\partial b} = \sum_{batch} (1 - tanh(W_{ax}x^{<t>} + W_{aa}a^{<t-1>} + b)^2) = \sum_{batch} \partial tanh(x) $

$ \frac{\partial a^{<t>}}{\partial x^{<t>}} = W_{ax}^{T}(1 - tanh(W_{ax}x^{<t>} + W_{aa}a^{<t-1>} + b)^2) = W_{ax}^{T} \partial tanh(x) $

$ \frac{\partial a^{<t>}}{\partial a^{<t-1>}} = W_{aa}^{T}(1 - tanh(W_{ax}x^{<t>} + W_{aa}a^{<t-1>} + b)^2) = W_{aa}^{T} \partial tanh(x)x^{<t>T} $

In [39]:
def rnn_cell_backward(da_next, cache):
    """
    Implement the backward pass for the RNN-cell.
    
    Arguments:
        da_next -- Gradient of loss with respect to next hidden-state.
        cache -- python dictionary containing useful values.
        
    Returns:
        gradients -- Python dictionary containing:
            dx -- Gradients of input data, numpy array of shape (n_x, m).
            da_prev -- Gradients of previous hidden-state, numpy array of shape (n_a, m).
            dWax -- Gradients of input-to-hidden weights, numpy array of shape (n_a, n_x).
            dWaa -- Gradients of hidden-to-hidden weights, numpy array of shape (n_a, n_a).
            dba -- Gradients of bias vector, numpy array of shape (n_a, 1).
    """
    
    # Retrieve values from cache.
    (a_next, a_prev, xt, params) = cache
    
    # Retrieve values from params.
    Wax = params["Wax"]
    Waa = params["Waa"]
    Wya = params["Wya"]
    ba = params["ba"]
    by = params["by"]
    
    dtanh = (1 - a_next * a_next) * da_next
    dWaa = np.dot(dtanh, a_prev.T)
    dWax = np.dot(dtanh, xt.T)
    dba = np.sum(dtanh, keepdims=True, axis=-1)

    dxt = np.dot(Wax.T, dtanh)
    da_prev = np.dot(Waa.T, dtanh)
    
    gradients = {
        "dxt": dxt,
        "da_prev": da_prev,
        "dWax": dWax,
        "dWaa": dWaa,
        "dba": dba
    }
    
    return gradients

In [40]:
np.random.seed(1)
xt = np.random.randn(3, 10)
a_prev = np.random.randn(5, 10)
Waa = np.random.randn(5, 5)
Wax = np.random.randn(5, 3)
Wya = np.random.randn(2, 5)
ba = np.random.randn(5, 1)
by = np.random.randn(2, 1)
parameters = {
    "Waa": Waa,
    "Wax": Wax,
    "Wya": Wya,
    "ba": ba,
    "by": by
}

a_next, yt_pred, cache = rnn_cell_forward(xt, a_prev, parameters)
# print("a_next[4] = {}".format(a_next[4]))
# print("a_next.shape = {}".format(a_next.shape))
# print("yt_pred[1] = {}".format(yt_pred[1]))
# print("yt_pred.shape = {}".format(yt_pred.shape))

da_next = np.random.randn(5, 10)
gradients = rnn_cell_backward(da_next, cache)
print("gradients[\"dxt\"][1][2] ={}".format(gradients["dxt"][1][2]))
print("gradients[\"dxt\"].shape ={}".format(gradients["dxt"].shape))
print("gradients[\"da_prev\"][2][3] ={}".format(gradients["da_prev"][2][3]))
print("gradients[\"da_prev\"].shape ={}".format(gradients["da_prev"].shape))
print("gradients[\"dWax\"][3][1] ={}".format(gradients["dWax"][3][1]))
print("gradients[\"dWax\"].shape ={}".format(gradients["dWax"].shape))
print("gradients[\"dWaa\"][1][2] ={}".format(gradients["dWaa"][1][2]))
print("gradients[\"dWaa\"].shape ={}".format(gradients["dWaa"].shape))
print("gradients[\"dba\"][4] ={}".format(gradients["dba"][4]))
print("gradients[\"dba\"].shape ={}".format(gradients["dba"].shape))

gradients["dxt"][1][2] =1.3653821219712916
gradients["dxt"].shape =(3, 10)
gradients["da_prev"][2][3] =-0.04357779106461625
gradients["da_prev"].shape =(5, 10)
gradients["dWax"][3][1] =-1.5012584841864745
gradients["dWax"].shape =(5, 3)
gradients["dWaa"][1][2] =1.1441951795389382
gradients["dWaa"].shape =(5, 5)
gradients["dba"][4] =[1.42397243]
gradients["dba"].shape =(5, 1)


### RNN forward

In [5]:
def rnn_forward(x, a0, params):
    """
    Implement the forward propagation of the recurrent neural network.
    
    Arguments:
        x -- Input data for every time-step, numpy array of shape (n_x, m, T_x).
        a0 -- Initial hidden-state, numpy array of shape (n_a, m).
        params -- python dictionary containing:
            Waa -- Weight matrix multiplying the hidden-state, numpy array of shape (n_a, n_a).
            Wax -- Weight matrix multiplying the input, numpy array of shape (n_a, n_x).
            Wya -- Weight matrix relating the hidden-state to the output, numpy array of shape (n_y, n_a).
            ba -- Bias, numpy array of shape (n_a, 1).
            by -- Bias relating the hidden-state to the output, numpy array of shape (n_y, 1).
            
    Returns:
        a -- Hidden-states for every time-step, numpy array of shape (n_a, m, T_x).
        y_pred -- Predictions for every time-step, numpy array of shape (n_y, m, T_x).
        caches -- Tuple of values needed for the backward pass, contains (list of caches, x)
    """
    
    # Initialize "caches" which will contain the list of all caches
    caches = []
    
    # Retrieve dimensions from shapes of x and Wy
    n_x, m, T_x = x.shape
    n_y, n_a = params["Wya"].shape
    
    # Initialize "a" and "y" with zeros
    a = np.zeros((n_a, m , T_x))
    y_pred = np.zeros((n_y, m , T_x))
    
    # Initialize a_next
    a_next = a0
    
    for t in range(T_x):
        a_next, yt_pred, cache = rnn_cell_forward(x[:, :, t], a_next, params)
        a[:, :, t] = a_next
        y_pred[:, :, t] = yt_pred
        caches.append(cache)
        
    caches = (caches, x)
    
    return a, y_pred, caches
    

In [13]:
np.random.seed(1)
x = np.random.randn(3, 10, 4)
a0 = np.random.randn(5, 10)
Waa = np.random.randn(5, 5)
Wax = np.random.randn(5, 3)
Wya = np.random.randn(2, 5)
ba = np.random.randn(5, 1)
by = np.random.randn(2, 1)

parameters = {
    "Waa": Waa,
    "Wax": Wax,
    "Wya": Wya,
    "ba": ba,
    "by": by
}

a, y_pred, caches = rnn_forward(x, a0, parameters)
print("a[4][1] = {}".format(a[4][1]))
print("a.shape = {}".format(a.shape))
print("y_pred[1][3] = {}".format(y_pred[1][3]))
print("y_pred.shape = {}".format(y_pred.shape))
print("caches[1][1][3] = {}".format(caches[1][1][3]))
print("len(caches) = {}".format(len(caches)))

a[4][1] = [-0.99999375  0.77911235 -0.99861469 -0.99833267]
a.shape = (5, 10, 4)
y_pred[1][3] = [0.79560373 0.86224861 0.11118257 0.81515947]
y_pred.shape = (2, 10, 4)
caches[1][1][3] = [-1.1425182  -0.34934272 -0.20889423  0.58662319]
len(caches) = 2


### RNN backward

In [49]:
def rnn_backward(da, caches):
    """
    Implement the backward pass for a RNN over an entire sequence of input data.
    
    Arguments:
        da --  Upstream gradients of all hidden-states, numpy array of shape (n_a, m, T_x).
        caches -- Tuple containing information from the forward pass
        
    Returns:
        gradients -- Python dictionary containing:
            dx -- Gradient w.r.t the input data, numpy array of shape (n_x, m, T_x).
            da0 -- Gradient w.r.t the initial hidden-state, numpy array of shape (n_a, m).
            dWax -- Gradient w.r.t the input's weight matrix, numpy array of shape (n_a, n_x).
            dWaa -- Gradient w.r.t the hidden-state's weight matrix, numpy array of shape (n_a, n_a).
            dba -- Gradient w.r.t the bias, numpy array of shape (n_a, 1)
    """
    
    (caches, x) = caches
    (a1, a0, x1, params) = caches[0]
    
    n_a, m, T_x = da.shape
    n_x, m = x1.shape
    
    dx = np.zeros((n_x, m, T_x))
    dWax = np.zeros((n_a, n_x))
    dWaa = np.zeros((n_a, n_a))
    dba = np.zeros((n_a, 1))
    da0 = np.zeros((n_a, m))
    da_prevt =np.zeros((n_a, m))
    
    for t in reversed(range(T_x)):
        gradients = rnn_cell_backward(da[:, :, t] + da_prevt, caches[t])
        dxt, da_prevt, dWaxt, dWaat, dbat = gradients["dxt"], gradients["da_prev"], gradients["dWax"], gradients["dWaa"], gradients["dba"]
        dx[:, :, t] = dxt
        dWax += dWaxt
        dWaa += dWaat
        dba += dbat
    
    da0 = da_prevt
    
    gradients = {
        "dx": dx,
        "da0": da0,
        "dWax": dWax,
        "dWaa": dWaa,
        "dba": dba
    }
    return gradients

In [50]:
np.random.seed(1)
x = np.random.randn(3, 10, 4)
a0 = np.random.randn(5, 10)
Waa = np.random.randn(5, 5)
Wax = np.random.randn(5, 3)
Wya = np.random.randn(2, 5)
ba = np.random.randn(5, 1)
by = np.random.randn(2, 1)

parameters = {
    "Waa": Waa,
    "Wax": Wax,
    "Wya": Wya,
    "ba": ba,
    "by": by
}

a, y_pred, caches = rnn_forward(x, a0, parameters)
# print("a[4][1] = {}".format(a[4][1]))
# print("a.shape = {}".format(a.shape))
# print("y_pred[1][3] = {}".format(y_pred[1][3]))
# print("y_pred.shape = {}".format(y_pred.shape))
# print("caches[1][1][3] = {}".format(caches[1][1][3]))
# print("len(caches) = {}".format(len(caches)))

da = np.random.randn(5, 10, 4)
gradients = rnn_backward(da, caches)
print("gradients[\"dx\"][1][2] ={}".format(gradients["dx"][1][2]))
print("gradients[\"dx\"].shape ={}".format(gradients["dx"].shape))
print("gradients[\"da0\"][2][3] ={}".format(gradients["da0"][2][3]))
print("gradients[\"da0\"].shape ={}".format(gradients["da0"].shape))
print("gradients[\"dWax\"][3][1] ={}".format(gradients["dWax"][3][1]))
print("gradients[\"dWax\"].shape ={}".format(gradients["dWax"].shape))
print("gradients[\"dWaa\"][1][2] ={}".format(gradients["dWaa"][1][2]))
print("gradients[\"dWaa\"].shape ={}".format(gradients["dWaa"].shape))
print("gradients[\"dba\"][4] ={}".format(gradients["dba"][4]))
print("gradients[\"dba\"].shape ={}".format(gradients["dba"].shape))

gradients["dx"][1][2] =[-0.86050481 -0.14439617 -0.02986862  0.10659932]
gradients["dx"].shape =(3, 10, 4)
gradients["da0"][2][3] =0.005796914346531472
gradients["da0"].shape =(5, 10)
gradients["dWax"][3][1] =0.44296398799038283
gradients["dWax"].shape =(5, 3)
gradients["dWaa"][1][2] =0.4418386736206451
gradients["dWaa"].shape =(5, 5)
gradients["dba"][4] =[-3.51296232]
gradients["dba"].shape =(5, 1)
